In [1]:
# The neccessary packages
## In-house packages
from saveload import save,load
## Pacakages from anywhere else
import os
from os import path as osp 
import netCDF4 as nc
from netCDF4 import Dataset
import numpy as np
import pandas as pd
import rasterio
from rasterio.transform import rowcol, xy
from pyproj import CRS, Transformer

In [2]:
# The file paths
## Define the base directory (main)
main = osp.join('C:/', 'Users', 'T-Spe', 'OneDrive', 'School', "Fall '25", "Master's Project", 'test')
tifdata = osp.join(main, 'tifdata')

## topography paths
elevation_path = osp.join(main, 'LH20_Elev_220.tif')

## fire detection (red pixel detection....)
fire_path = osp.join(tifdata, 'ml_data')

# row, col indices and mask file path
rowcol_test_path = osp.join(main, 'row_col_and_mask_sample.nc')

In [3]:
def create_netcdf_with_row_col_and_mask(
    lon_array,
    lat_array,
    raster_crs,
    transform,
    raster_shape,
    output_file,
    debug=False
):
    """
    Create a NetCDF file with row, column indices, and a spatial mask for provided lon/lat data.
    
    Args:
        lon_array (np.ndarray): Array of longitudes in WGS84.
        lat_array (np.ndarray): Array of latitudes in WGS84.
        raster_crs (str): CRS of the raster (e.g., "EPSG:5070").
        transform (Affine): Rasterio affine transform of the raster.
        raster_shape (tuple): Shape of the raster (rows, cols).
        output_file (str): Path to the output NetCDF file.
        debug (bool): Whether to enable debug messages.
    """
    print(f"Creating NetCDF file at {output_file}...")

    # Initialize transformer
    transformer = Transformer.from_crs("EPSG:4326", raster_crs, always_xy=True)

    # Reproject lon/lat to raster CRS
    print("Transforming coordinates to raster CRS...")
    raster_lon, raster_lat = transformer.transform(lon_array, lat_array)

    # Calculate row and column indices
    inv_transform = ~transform
    cols, rows = inv_transform * (raster_lon, raster_lat)

    # Round indices and convert to integers
    rows = np.round(rows).astype(int)
    cols = np.round(cols).astype(int)

    # Create a valid mask based on raster shape
    valid_mask = (
        (rows >= 0) & (rows < raster_shape[0]) &
        (cols >= 0) & (cols < raster_shape[1])
    )
    rows_valid = rows[valid_mask]
    cols_valid = cols[valid_mask]

    if debug:
        print(f"Valid row indices: {rows_valid}")
        print(f"Valid col indices: {cols_valid}")
        print(f"Valid mask shape: {valid_mask.shape}")
        print(f"Number of valid points: {np.sum(valid_mask)}")

    # Write to NetCDF
    with Dataset(output_file, 'w', format='NETCDF4') as nc_file:
        # Define dimensions
        nc_file.createDimension('points', len(rows_valid))
        nc_file.createDimension('mask', len(valid_mask))

        # Create variables
        rows_var = nc_file.createVariable('rows', 'i4', ('points',), zlib=True)
        cols_var = nc_file.createVariable('cols', 'i4', ('points',), zlib=True)
        mask_var = nc_file.createVariable('valid_mask', 'i1', ('mask',), zlib=True)

        # Write data
        rows_var[:] = rows_valid
        cols_var[:] = cols_valid
        mask_var[:] = valid_mask.astype(int)

    print(f"NetCDF file saved: {output_file}")

def validate_row_col(rows, cols, raster_crs, transform):
    """
    Validate the row and column indices by reprojecting them back to longitude and latitude.

    Args:
        rows (np.ndarray): Array of row indices.
        cols (np.ndarray): Array of column indices.
        raster_crs (str): CRS of the raster (e.g., "EPSG:5070").
        transform (Affine): Rasterio affine transform of the raster.

    Returns:
        tuple: Reprojected longitude and latitude arrays.
    """
    print("Validating row and column indices by reprojecting back to longitude and latitude...")

    # Use rasterio's transform to compute the coordinates
    raster_coords = xy(transform, rows, cols, offset='center')

    # Extract the projected coordinates
    raster_x, raster_y = raster_coords

     # Step 2: Reproject from the raster CRS to WGS84
    transformer = Transformer.from_crs(raster_crs, "EPSG:4326", always_xy=True)
    lon_wgs84, lat_wgs84 = transformer.transform(raster_x, raster_y)

    #print(f"First 10 Reprojected WGS84 Longitudes: {lon_wgs84[:10]}")
    #print(f"First 10 Reprojected WGS84 Latitudes: {lat_wgs84[:10]}")

    # Verify if any NaN or invalid values exist
    if np.isnan(lon_wgs84).any() or np.isnan(lat_wgs84).any():
        raise ValueError("Reprojected coordinates contain NaN values.")

    return lon_wgs84, lat_wgs84

In [4]:
def load_fire_detection(file_path, confidence_threshold):
    """
    Load and process fire detection data.
    Retains all points but filters out those with a label of 1 and confidence < confidence_threshold.

    """
    print("Loading fire detection data...")
    X, y, c, basetime = load(file_path)

    # Debug: Print initial statistics
    print(f"Total data points: {len(X)}")
    print(f"Number of 'Fire' labels: {np.sum(y == 1)}")
    print(f"Number of 'Fire' labels with confidence < {confidence_threshold}: {np.sum((y == 1) & (c < confidence_threshold))}")
    
    # Filter out points with label 1 and confidence < confidence_threshold
    valid_indices = ~((y == 1) & (c < confidence_threshold))  # Keep points not failing this condition
    X_filtered = X[valid_indices]
    y_filtered = y[valid_indices]

    # Debug: Print post-filtering statistics
    print(f"Number of remaining data points: {len(X_filtered)}")
    print(f"Number of remaining 'Fire' labels: {np.sum(y_filtered == 1)}")

    # Extract filtered components
    lon_array = X_filtered[:, 0]
    lat_array = X_filtered[:, 1]
    time_in_days = X_filtered[:, 2]
    dates_fire_actual = basetime + pd.to_timedelta(time_in_days, unit='D')
    dates_fire = dates_fire_actual.floor("h")  # Round to nearest hour

    print(f"Loaded {len(X)} data points, filtered down to {len(X_filtered)} based on confidence and labels.")
    
    return {
        "lon": lon_array,
        "lat": lat_array,
        "time_days": time_in_days,
        "dates_fire": dates_fire,
        "labels": y_filtered
    }
    


In [5]:
#### CHECK THE CRS OF THE FILES ##### (Complete, but maybe need to make a script to make it more formal)
## topography
elevation_dataset = rasterio.open(elevation_path)
elevation = elevation_dataset.read(1)
transform = elevation_dataset.transform
raster_crs = elevation_dataset.crs.to_string()
raster_shape = elevation.shape

# Load fire detection data
fire_detection_data = load_fire_detection(fire_path, confidence_threshold=70)
lon_array = fire_detection_data['lon']
lat_array = fire_detection_data['lat']
dates_fire = fire_detection_data['dates_fire']
labels = fire_detection_data['labels']

Loading fire detection data...
Total data points: 1343281
Number of 'Fire' labels: 14
Number of 'Fire' labels with confidence < 70: 10
Number of remaining data points: 1343271
Number of remaining 'Fire' labels: 4
Loaded 1343281 data points, filtered down to 1343271 based on confidence and labels.


In [6]:
# Sample data and parameters
# lon_array_test = np.array([-155.3, -155.5, -155.8])
# lat_array_test = np.array([19.6, 19.7, 19.8])
output_netcdf = osp.join(main, "row_col_and_mask_sample.nc")
debug_mode = True

# Call the function to create the saved file with row, col indices, and mask for spatial points outside of raster
# NOTE: The saved file is there so this block does not need to be ran again.
create_netcdf_with_row_col_and_mask(
    lon_array,
    lat_array,
    raster_crs,
    transform,
    raster_shape,
    output_netcdf,
    debug=debug_mode
)


Creating NetCDF file at C:/Users\T-Spe\OneDrive\School\Fall '25\Master's Project\test\row_col_and_mask_sample.nc...
Transforming coordinates to raster CRS...
Valid row indices: [5219 5231 5243 ... 3422 3396 3411]
Valid col indices: [4557 4497 4436 ... 3345 3334 3342]
Valid mask shape: (1343271,)
Number of valid points: 1162036
NetCDF file saved: C:/Users\T-Spe\OneDrive\School\Fall '25\Master's Project\test\row_col_and_mask_sample.nc


In [7]:
# Read the saved test file, check the shapes and minor data exploration
row_col_data = nc.Dataset(rowcol_test_path)

# Check shapes
mask_shape = row_col_data.variables['valid_mask'][:].shape
rows_shape = row_col_data.variables['rows'][:].shape
cols_shape = row_col_data.variables['cols'][:].shape

print(f"The shape of the mask array is: {mask_shape}")
print(f"The shape of the row array is: {rows_shape}")
print(f"The shape of the column array is: {cols_shape}")

# Check min, max, and NaN values for 'valid_mask'
valid_mask = row_col_data.variables['valid_mask'][:]
print(f"Mask min: {valid_mask.min()}, Mask max: {valid_mask.max()}")
print(f"Mask contains NaNs: {np.isnan(valid_mask).any()}")
print(f"Mask contains None: {np.any(valid_mask == None)}")  # NaNs should suffice, but adding for clarity

# Check min, max, and NaN values for 'rows'
rows = row_col_data.variables['rows'][:]
print(f"Rows min: {rows.min()}, Rows max: {rows.max()}")
print(f"Rows contains NaNs: {np.isnan(rows).any()}")
print(f"Rows contains None: {np.any(rows == None)}")

# Check min, max, and NaN values for 'cols'
cols = row_col_data.variables['cols'][:]
print(f"Columns min: {cols.min()}, Columns max: {cols.max()}")
print(f"Columns contain NaNs: {np.isnan(cols).any()}")
print(f"Columns contain None: {np.any(cols == None)}")

The shape of the mask array is: (1343271,)
The shape of the row array is: (1162036,)
The shape of the column array is: (1162036,)
Mask min: 0, Mask max: 1
Mask contains NaNs: False
Mask contains None: False
Rows min: 0, Rows max: 5257
Rows contains NaNs: False
Rows contains None: False
Columns min: 0, Columns max: 4608
Columns contain NaNs: False
Columns contain None: False


In [8]:
# Reproject the (row,col) back to (lon, lat) and compare to (lon, lat) used to obtain (row, col)
valid_mask = row_col_data.variables['valid_mask'][:].astype(bool)
lon_array_valid = lon_array[valid_mask]
lat_array_valid = lat_array[valid_mask]

lon_reproj_row , lat_reproj_col = validate_row_col(rows, cols, raster_crs, transform)

diff_lon = np.abs(lon_reproj_row - lon_array_valid)
diff_lat = np.abs(lat_reproj_col - lat_array_valid)
print(f"Max Longitude Difference: {np.max(diff_lon)}")
print(f"Max Latitude Difference: {np.max(diff_lat)}")


Validating row and column indices by reprojecting back to longitude and latitude...
Max Longitude Difference: 0.000287893844415521
Max Latitude Difference: 0.00027189029520258146


# Previous implentation of obtaining row and column indices, testing the values obtain on sampled Non-barren points

In [ ]:
from pyproj import Transformer
import numpy as np
import random
import rasterio
from rasterio.transform import rowcol,xy

def get_row_col(lon_array, lat_array, raster_crs, transform, raster_shape, debug):
    """
    Optimized function to compute row and column indices for arrays of longitudes and latitudes.

    Args:
        lon_array (np.ndarray): Array of longitudes in WGS84.
        lat_array (np.ndarray): Array of latitudes in WGS84.
        raster_crs (str): CRS of the raster (e.g., "EPSG:5070").
        transform (Affine): Rasterio affine transform of the raster.

    Returns:
        tuple: Arrays of row and column indices in the raster.
    """
    print('Computing row and column indices for topography and vegetation files...')

    # Reproject lon/lat arrays to the raster's CRS using pyproj
    transformer = Transformer.from_crs("EPSG:4326", raster_crs, always_xy=True)

    if debug:
        print(f"Debug: The raster CRS is: {raster_crs}")
        print(f"Debug: The raster shape is: {raster_shape}")
        print(f"Debug: The transform for the raster is: {transform}")
        print(f"Debug: lon_array shape: {lon_array.shape}, lat_array shape: {lat_array.shape}")
        print(f"Debug: lon_array: {lon_array}")
        print(f"Debug: lat_array: {lat_array}")
        print(f"Debug: NaNs in lon_array: {np.isnan(lon_array).any()}, NaNs in lat_array: {np.isnan(lat_array).any()}")
        print(f"Debug: lon_array min/max: {lon_array.min()} / {lon_array.max()}")
        print(f"Debug: lat_array min/max: {lat_array.min()} / {lat_array.max()}")
        print("Starting coordinate transformation...")  # Before transformer is built
        try:
            raster_lon, raster_lat = transformer.transform(lon_array[0], lat_array[0])
            print(f"Debug: Single-point transformation successful: {raster_lon}, {raster_lat}")
        except Exception as e:
            print(f"Debug: Error during single-point transformation: {e}")

    print("Transforming coordinates to raster CRS...")  # Before transformation
    raster_lon, raster_lat = transformer.transform(lon_array, lat_array)
    if debug:
        print("Coordinate transformation completed.")  # After transformation
        print("Starting affine transformation for row/col computation...")  # Before affine transformation

    # Calculate row and column indices using vectorized transformation
    inv_transform = ~transform
    cols, rows = inv_transform * (raster_lon, raster_lat)
    print(f"The affine transformation is complete and the row,col values have been extracted...")

    # Round to nearest integer and convert to int
    rows = np.round(rows).astype(int)
    cols = np.round(cols).astype(int)

    if debug:
        # Debugging: Check bounds, reprojected coordinates and other metrics
        print(f"Debug: WGS84 lon min/max (pre-mask): {lon_array.min()} / {lon_array.max()}")
        print(f"Debug: WGS84 lat min/max (pre-mask): {lat_array.min()} / {lat_array.max()}")
        print(f"Debug: Reprojected lon min/max (pre-mask): {raster_lon.min()} / {raster_lon.max()}")
        print(f"Debug: Reprojected lat min/max (pre-mask): {raster_lat.min()} / {raster_lat.max()}")
        print(f"Debug: Rows min/max(pre-mask): {rows.min()}, {rows.max()}")
        print(f"Debug: Cols min/max(pre-mask): {cols.min()}, {cols.max()}")
        print(f"Debug: The shape of rows, cols: {rows.shape, cols.shape}")

    print("Truncating rows, cols, lon_array, and lat_array based on valid raster array inputs...")

    # Create a mask to filter valid row/col indices
    rowcol_mask = (
            (rows >= 0) & (rows < raster_shape[0]) &
            (cols >= 0) & (cols < raster_shape[1])
    )

    # Calculate bounds in raster CRS
    raster_x_min, raster_y_min = transform * (0, raster_shape[0])
    raster_x_max, raster_y_max = transform * (raster_shape[1], 0)

    # Reproject bounds to WGS84
    transformer = Transformer.from_crs(raster_crs, "EPSG:4326", always_xy=True)

    lon_min, lat_min = transformer.transform(raster_x_min, raster_y_min)
    lon_max, lat_max = transformer.transform(raster_x_max, raster_y_max)

    # Create a valid mask based on coordinate bounds via the raster boundaries
    coord_mask = (
            (lon_array >= lon_min) & (lon_array <= lon_max) &
            (lat_array >= lat_min) & (lat_array <= lat_max)
    )

    # Combine the masks
    valid_mask = rowcol_mask & coord_mask

    # Apply the mask and filter spatial data accordingly
    rows_valid = rows[valid_mask]
    cols_valid = cols[valid_mask]
    lon_array_valid = lon_array[valid_mask]
    lat_array_valid = lat_array[valid_mask]

    if debug:
        print(f"Raster bounds in WGS84: lon_min={lon_min}, lon_max={lon_max}, lat_min={lat_min}, lat_max={lat_max}")
        print(f"Debug: WGS84 lon min/max (post-mask): {lon_array_valid.min()} / {lon_array_valid.max()}")
        print(f"Debug: WGS84 lat min/max (post-mask): {lat_array_valid.min()} / {lat_array_valid.max()}")
        print(f"Debug: Reprojected lon min/max (post-mask): {raster_x_min} / {raster_x_max}")
        print(f"Debug: Reprojected lat min/max (post-mask): {raster_y_min} / {raster_y_max}")
        print(f"Debug: Rows min/max(post-mask): {rows_valid.min()}, {rows_valid.max()}")
        print(f"Debug: Cols min/max(post-mask): {cols_valid.min()}, {cols_valid.max()}")

    return rows_valid, cols_valid, lon_array_valid, lat_array_valid, valid_mask

def sample_non_barren_points(lon_array, lat_array, fuelmod, num_samples, elevation_transform, raster_crs, raster_shape):
    """
    Sample points from non-barren areas in the fuelmod raster and test both workflows.

    Args:
        lon_array (np.ndarray): Array of longitudes.
        lat_array (np.ndarray): Array of latitudes.
        fuelmod (np.ndarray): Categorical raster data (e.g., vegetation types).
        num_samples (int): Number of points to sample.
        elevation_transform: Rasterio affine transform for the elevation raster.
        raster_crs (str): CRS of the raster (e.g., EPSG code).

    Returns:
        None
    """
    print("Filtering non-barren areas...")

    # Identify non-barren areas
    non_barren_mask = fuelmod != "Barren"
    non_barren_indices = np.argwhere(non_barren_mask)  # Returns a 2D array of (row, col) pairs

    if len(non_barren_indices) == 0:
        print("No non-barren areas found in the raster.")
        return

    if len(non_barren_indices) < num_samples:
        print(f"Warning: Only {len(non_barren_indices)} non-barren areas found, adjusting sample size.")
        num_samples = len(non_barren_indices)

    # Randomly sample from non-barren indices
    sampled_indices = random.sample(list(non_barren_indices), num_samples)

    for idx, (row, col) in enumerate(sampled_indices):
        try:
            # Convert row, col to lon, lat for testing reverse workflow
            lon, lat = validate_row_col(row, col, raster_crs, elevation_transform)
            print(f"\nTesting sample {idx + 1}/{num_samples}")
            print(f"Row: {row}, Col: {col}, Converted lon: {lon}, lat: {lat}")

            # Convert lon, lat back to row, col to validate consistency
            test_row, test_col ,test_lon, test_lat, spatial_mask= get_row_col(np.array([lon]), np.array([lat]), raster_crs, elevation_transform, elevation.shape, False)
            print(f"Original row: {row}, Original col: {col}")
            print(f"Recomputed row: {test_row}, Recomputed col: {test_col}")

            # Validate raster indexing
            if 0 <= row < fuelmod.shape[0] and 0 <= col < fuelmod.shape[1]:
                fuelmod_val = fuelmod[row, col]
                print(f"Fuelmod value at sampled point: {fuelmod_val}")
            else:
                print(f"Out of bounds for fuelmod raster. Row={row}, Col={col}.")
        except Exception as e:
            print(f"Error encountered for sample {idx + 1}/{num_samples}: {e}")


In [ ]:
# Example usage
sample_non_barren_points(
        lon_array=lon_array,
        lat_array=lat_array,
        fuelmod=fuelmod,
        num_samples=10,
        elevation_transform=fuelmod_dataset.transform,
        raster_crs=fuelmod_dataset.crs.to_string(),
        raster_shape=elevation.shape
    )


In [ ]:
# Build the interpolator
interp = Coord_to_index(degree = 2)
interp.build(lon_grid, lat_grid) 

def calc_rhum(temp_K, mixing_ratio):
    try:
        # Constants
        epsilon = 0.622
        pressure_pa = 1000 * 100 # fixed until we can get the data
        
        # Saturation vapor pressure
        es_hpa = 6.112 * np.exp((17.67 * (temp_K - 273.15)) / ((temp_K - 273.15) + 243.5))
        es_pa = es_hpa * 100
        
        # Actual vapor pressure
        e_pa = (mixing_ratio * pressure_pa) / (epsilon + mixing_ratio)
        
        # Relative humidity
        rh = (e_pa / es_pa) * 100
        return rh
    except Exception as e:
        print(f"Error in calc_rhum: temp_K={temp_K}, mixing_ratio={mixing_ratio}, error={e}")
        return np.nan

def interpolate_data(lon_array, lat_array, date, interp, raster_crs, transform, raster_shape, elevation, slope, aspect, fuelmod, temp, rain, vapor, wind_u, wind_v, debug):
    """
    Interpolate meteorological data and extract raster values for static features (batch processing).

    Args:
        lon_array (np.ndarray): Array of longitudes in WGS84.
        lat_array (np.ndarray): Array of latitudes in WGS84.
        date (datetime): Date associated with the interpolation.
        interp: Interpolator object for meteorological data.
        transform (Affine): Rasterio affine transform for static rasters.
        raster_crs (str): CRS of the raster (e.g., "EPSG:5070").
        elevation, slope, aspect, fuelmod: Static raster arrays.
        temp, rain, vapor, wind_u, wind_v: Meteorological arrays.

    Returns:
        list: List of dictionaries with interpolated meteorological data and static raster features.
    """
    results = []

    # Precompute rows and cols
    rows, cols, lons, lats, mask = get_row_col(lon_array, lat_array, raster_crs, transform, raster_shape, debug)

    for idx, (lon, lat, row, col) in enumerate(zip(lon_array, lat_array, rows, cols)):
        try:
            # Interpolate meteorological data
            ia, ja = interp.evaluate(lon, lat)
            i, j = np.round(ia).astype(int), np.round(ja).astype(int)

            # Extract meteorological values
            temp_val = temp[i, j]
            rain_val = rain[i, j]
            rhum_val = calc_rhum(temp[i, j], vapor[i, j])
            wind_val = np.sqrt(wind_u[i, j]**2 + wind_v[i, j]**2)

            # Check bounds and extract static raster values
            if 0 <= row < elevation.shape[0] and 0 <= col < elevation.shape[1]:
                elevation_val = elevation[row, col]
                slope_val = slope[row, col]
                aspect_val = aspect[row, col]
                fuelmod_val = fuelmod[row, col]
            else:
                elevation_val, slope_val, aspect_val, fuelmod_val = np.nan, np.nan, np.nan, "Out of bounds"

            # Build output dictionary
            data_dict = {
                'date': date,
                'lon': lon,
                'lat': lat,
                'temp': temp_val,
                'rain': rain_val,
                'rhum': rhum_val,
                'wind': wind_val,
                'elevation': elevation_val,
                'slope': slope_val,
                'aspect': aspect_val,
                'fuelmod': fuelmod_val,
            }
            results.append(data_dict)

        except Exception as e:
            print(f"Error interpolating and extracting data for lon={lon}, lat={lat}: {e}")

    return results

# Example lon and lat arrays
lon_array = np.array([-155.345, -155.678, -155.789])
lat_array = np.array([19.123, 19.567, 19.890])
date = "2024-11-19"  # Example date

# Interpolate and extract data for all points
results = interpolate_data(
    lon_array=lon_array,
    lat_array=lat_array,
    date=date,
    interp=interp,
    raster_crs=elevation_dataset.crs.to_string(),
    transform=elevation_transform,
    raster_shape = elevation.shape,
    elevation=elevation,
    slope=slope,
    aspect=aspect,
    fuelmod=fuelmod,
    temp=temp,
    rain=rain,
    vapor=vapor,
    wind_u=wind_u,
    wind_v=wind_v,
    debug = False
)

# Print each result as a nicely formatted JSON
for result in results:
    converted_result = {key: (value.item() if isinstance(value, np.generic) else value) for key, value in result.items()}
    print(converted_result)

# Start timing and resource monitoring
start_time = time.time()
process = psutil.Process(os.getpid())
start_cpu = process.cpu_percent(interval=None)
start_mem = process.memory_info().rss  # in bytes

# Run parallel interpolation
data_interp = Parallel(n_jobs=-4)(
    delayed(interpolate_data)(lon, lat) for lon, lat in zip(lon_array, lat_array)
)

# Convert the list of dictionaries to a DataFrame for easy handling
df = pd.DataFrame(data_interp)

# End timing and resource monitoring
end_time = time.time()
end_cpu = process.cpu_percent(interval=None)
end_mem = process.memory_info().rss  # in bytes

# Calculate the differences
total_time = end_time - start_time
cpu_usage = end_cpu - start_cpu
memory_usage = end_mem - start_mem

print(f"Script runtime: {total_time:.2f} seconds")
print(f"CPU usage change: {cpu_usage:.2f}%")
print(f"Memory usage change: {memory_usage / (1024 ** 2):.2f} MB")

In [ ]:
# Start timing and resource monitoring
start_time = time.time()
process = psutil.Process(os.getpid())
start_cpu = process.cpu_percent(interval=None)
start_mem = process.memory_info().rss  # in bytes

# Run interpolation in a simple loop
data_interp = []
for lon, lat in zip(lon_array, lat_array):
    data_interp.append(interpolate_data(lon, lat))
    
# Convert the list of dictionaries to a DataFrame for easy handling
df = pd.DataFrame(data_interp)

# End timing and resource monitoring
end_time = time.time()
end_cpu = process.cpu_percent(interval=None)
end_mem = process.memory_info().rss  # in bytes

# Calculate the differences
total_time = end_time - start_time
cpu_usage = end_cpu - start_cpu
memory_usage = end_mem - start_mem

print(f"Script runtime: {total_time:.2f} seconds")
print(f"CPU usage change: {cpu_usage:.2f}%")
print(f"Memory usage change: {memory_usage / (1024 ** 2):.2f} MB")

In [ ]:
interpolate_data(lon_array[3], lat_array[5]) 

In [ ]:
print(df.head())

In [ ]:
# Feature engineering of meteorology variables (i.e. averaging to determine effect on fuel moisture)
print(dates_fire.max())